In [2]:
import numpy as np
import pandas as pd
import string 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#systeme
import sys
sys.setrecursionlimit(200000)

#nltk
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import ngrams

import warnings; warnings.simplefilter('ignore')

import re

import spacy
#from spacy_lefff import lefffLemmatizer

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

import fr_core_news_sm
nlp = fr_core_news_sm.load()

#%load_ext Cython


#wolf
#from frenetic.packet import *
#fwn = FreNetic("C:/Users/Utilisateur/Desktop/simplon/projetA1/wolf.xml") 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Tweets

In [3]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [7]:
tweets = pd.read_csv("./french-tweets-politique/tweets_unique.csv")

In [8]:
tweets.head()

,Unnamed: 0,date,text,hashtags
0,0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NaN
1,1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NaN
2,2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NaN
3,3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NaN
4,4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"['Macron', 'Bellatar', 'ConseilDesVilles']"


In [10]:
tweets.rename(columns={'Unnamed: 0':'num'}, inplace=True)

In [11]:
tweets.head()

,date,text,hashtags
0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NaN
1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NaN
2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NaN
3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NaN
4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"['Macron', 'Bellatar', 'ConseilDesVilles']"


In [12]:
tw = tweets.copy()

# Normalisation

In [13]:
low = lambda x: str(x).lower()
tweets['text'] = tweets.text.apply(low)
tweets

,date,text,hashtags
0,Sun Mar 18 13:45:13 +0000 2018,au lieu d'expulser des diplomates russes il eu...,NaN
1,Sun Mar 18 15:47:19 +0000 2018,"selon l’ex ambassadeur anglais en ouzbékistan,...",NaN
2,Sun Mar 18 11:48:32 +0000 2018,@philipj37 @eugeniebastie tellement vrai.\r\ns...,NaN
3,Sun Mar 18 15:41:24 +0000 2018,"bombe: le capitaine barril, ancien commandant ...",NaN
4,Sun Mar 18 14:32:39 +0000 2018,dans la même semaine #macron nomme le très amb...,"['Macron', 'Bellatar', 'ConseilDesVilles']"
5,Sat Mar 17 18:07:58 +0000 2018,une belle analyse concernant poutine ..\r\nno...,NaN
6,Sun Mar 18 15:47:11 +0000 2018,des irrégularités signalées lors de la préside...,NaN
7,Sat Mar 17 16:05:27 +0000 2018,être ami et être en couple avec quelqu'un ce s...,NaN
8,Sun Mar 18 15:47:10 +0000 2018,le contraire aurait été étonnant ... https://t...,NaN
9,Sun Mar 18 12:49:29 +0000 2018,notre correspondante en #russie @mandraud rapp...,['Russie']


In [14]:
tweets['text'] = tweets['text'].apply(lambda x: re.sub('[!@#$:’).";,?&%=>(<''©-]', '', x.lower()))
tweets['text'] = tweets['text'].apply(lambda x: re.sub('  ', ' ', x))
tweets['text']

0       au lieu d'expulser des diplomates russes il eu...
1       selon lex ambassadeur anglais en ouzbékistan l...
2       philipj37 eugeniebastie tellement vrai\r\ns'oc...
3       bombe le capitaine barril ancien commandant du...
4       dans la même semaine macron nomme le très ambi...
5        une belle analyse concernant poutine \r\nnos ...
6       des irrégularités signalées lors de la préside...
7       être ami et être en couple avec quelqu'un ce s...
8       le contraire aurait été étonnant https//tco/ej...
9       notre correspondante en russie mandraud rappor...
10      mormach cheyenne8304 agent de poutine spoutnik...
11      si le combat est inévitable il faut frapper le...
12      les images surréalistes partagées par l'opposa...
13      quand un média russe se fiche ouvertement du p...
14      gérard depardieu a voté pour la présidentielle...
15      poutine  4 ème mandat   dictateur\r\nmerkel   ...
16      vladimir poutine sa longévité au pouvoir encor...
17      avant 

In [15]:
for i in range(len(tweets['text'])):
    tweets['text'][i] = " ".join([word for word in tweets['text'][i].split()
                                if 'http' not in word and '@' not in word and '<' not in word])
tweets['text']

0       au lieu d'expulser des diplomates russes il eu...
1       selon lex ambassadeur anglais en ouzbékistan l...
2       philipj37 eugeniebastie tellement vrai s'occup...
3       bombe le capitaine barril ancien commandant du...
4       dans la même semaine macron nomme le très ambi...
5       une belle analyse concernant poutine nos polit...
6       des irrégularités signalées lors de la préside...
7       être ami et être en couple avec quelqu'un ce s...
8                        le contraire aurait été étonnant
9       notre correspondante en russie mandraud rappor...
10      mormach cheyenne8304 agent de poutine spoutnik...
11      si le combat est inévitable il faut frapper le...
12      les images surréalistes partagées par l'opposa...
13      quand un média russe se fiche ouvertement du p...
14      gérard depardieu a voté pour la présidentielle...
15      poutine 4 ème mandat dictateur merkel 4 ème ma...
16      vladimir poutine sa longévité au pouvoir encor...
17      avant 

# Tokénisation

In [16]:
tweets.dtypes

date        object
text        object
hashtags    object
dtype: object

In [17]:
tweets['text'].astype('str', inplace=True)

0       au lieu d'expulser des diplomates russes il eu...
1       selon lex ambassadeur anglais en ouzbékistan l...
2       philipj37 eugeniebastie tellement vrai s'occup...
3       bombe le capitaine barril ancien commandant du...
4       dans la même semaine macron nomme le très ambi...
5       une belle analyse concernant poutine nos polit...
6       des irrégularités signalées lors de la préside...
7       être ami et être en couple avec quelqu'un ce s...
8                        le contraire aurait été étonnant
9       notre correspondante en russie mandraud rappor...
10      mormach cheyenne8304 agent de poutine spoutnik...
11      si le combat est inévitable il faut frapper le...
12      les images surréalistes partagées par l'opposa...
13      quand un média russe se fiche ouvertement du p...
14      gérard depardieu a voté pour la présidentielle...
15      poutine 4 ème mandat dictateur merkel 4 ème ma...
16      vladimir poutine sa longévité au pouvoir encor...
17      avant 

In [18]:
clean = lambda tx: word_tokenize(tx)
tweets.text = tweets.text.apply(clean)
tweets.text

0       [au, lieu, d'expulser, des, diplomates, russes...
1       [selon, lex, ambassadeur, anglais, en, ouzbéki...
2       [philipj37, eugeniebastie, tellement, vrai, s'...
3       [bombe, le, capitaine, barril, ancien, command...
4       [dans, la, même, semaine, macron, nomme, le, t...
5       [une, belle, analyse, concernant, poutine, nos...
6       [des, irrégularités, signalées, lors, de, la, ...
7       [être, ami, et, être, en, couple, avec, quelqu...
8                  [le, contraire, aurait, été, étonnant]
9       [notre, correspondante, en, russie, mandraud, ...
10      [mormach, cheyenne8304, agent, de, poutine, sp...
11      [si, le, combat, est, inévitable, il, faut, fr...
12      [les, images, surréalistes, partagées, par, l'...
13      [quand, un, média, russe, se, fiche, ouverteme...
14      [gérard, depardieu, a, voté, pour, la, préside...
15      [poutine, 4, ème, mandat, dictateur, merkel, 4...
16      [vladimir, poutine, sa, longévité, au, pouvoir...
17      [avant

# Stop-Words

In [19]:
stopW = stopwords.words('french')
#print(stopW)
filtered_sentence = [w for w in tweets['text'] if not w in stopW] 
# remove URLs, RTs, and twitter handles
for i in range(len(filtered_sentence)):
    filtered_sentence[i] =" ".join([word for word in filtered_sentence[i]
                               if 'http' not in word  and '<' not in word])
filtered_sentence

["au lieu d'expulser des diplomates russes il eut été plus concrets et plus pragmatiques de s'occuper des ressortissants info pakistanais présumés responsables de milliers de viols d'enfants à telford mme may a opté pour la facilité face à une monstruosité ethnique",
 "selon lex ambassadeur anglais en ouzbékistan le `` novitchok '' serait `` une arnaque '' comme les `` armes de destruction massives '' irakiennes le royaume u refuse tt échantillon à loiac organisation prévention armes chimiques de la haye skripal",
 "philipj37 eugeniebastie tellement vrai s'occuper de trump et de poutine c'est tellement plus lieu d'essayer d'enlever la paille dans l'œil de ton voisinenlève d'abord la poutre dans ton oeil",
 'bombe le capitaine barril ancien commandant du gign « jai la preuve de qui a tué litvinenko – ce nest pas la russie » via avic_sy',
 "dans la même semaine macron nomme le très ambigu sur l'islamisme bellatar au conseildesvilles et boycotte le stand russe au salondulivre2018 c'est à 

In [20]:
def filtration(tk: list ):#texte tokenisé sous format de liste
    l_filtree = []
    for word in tk:
        if word not in stopW:
            l_filtree.append(word)
    return l_filtree

In [21]:
filtered_sentence = tweets['text'].apply(filtration)
filtered_sentence

0       [lieu, d'expulser, diplomates, russes, plus, c...
1       [selon, lex, ambassadeur, anglais, ouzbékistan...
2       [philipj37, eugeniebastie, tellement, vrai, s'...
3       [bombe, capitaine, barril, ancien, commandant,...
4       [semaine, macron, nomme, très, ambigu, l'islam...
5       [belle, analyse, concernant, poutine, politica...
6       [irrégularités, signalées, lors, présidentiell...
7       [être, ami, être, couple, quelqu'un, 2, choses...
8                                   [contraire, étonnant]
9       [correspondante, russie, mandraud, rapporte, b...
10      [mormach, cheyenne8304, agent, poutine, spoutn...
11      [si, combat, inévitable, faut, frapper, premie...
12      [les, images, surréalistes, partagées, l'oppos...
13      [quand, média, russe, fiche, ouvertement, peti...
14      [gérard, depardieu, a, voté, présidentielle, r...
15      [poutine, 4, ème, mandat, dictateur, merkel, 4...
16      [vladimir, poutine, longévité, pouvoir, encore...
17      [avant

# Lemmatisation

In [22]:
#fonction de lemmatisation
def lemma(l:list):
    new_l = ''
    for word in l:
        new_l += ' '+word
    # strip retire les espaces au debut et a la fin de la chaine de caractere
    new_l = new_l.strip()
    n = nlp(new_l)
    # boucle sur n et applique la fonction lemma_ sur chacun des tokens
    x = []
    for z in n:
        x.append(z.lemma_)
    return x

In [23]:
tweetslemma = filtered_sentence.apply(lemma)

In [24]:
tweetslemma

0       [lieu, de, expulser, diplomate, russe, plus, c...
1       [selon, lex, ambassadeur, anglais, ouzbékistan...
2       [philipj37, eugeniebastie, tellement, vrai, si...
3       [bombe, capitaine, barril, ancien, commander, ...
4       [semaine, macron, nommer, très, ambigu, le, is...
5       [bel, analyse, concerner, poutin, politicard, ...
6       [irrégularité, signaler, lors, présidentielle,...
7       [être, ami, être, coupler, quelqu'un, 2, chose...
8                                   [contraire, étonnant]
9       [correspondant, russie, mandraud, rapporter, b...
10      [mormach, cheyenne8304, agent, poutine, spoutn...
11      [si, combat, inévitable, falloir, frapper, pre...
12      [le, image, surréaliste, partagée, le, opposer...
13      [quand, média, russe, fich, ouvertemer, petit,...
14      [gérard, depardieu, avoir, voter, présidentiel...
15      [poutine, 4, ème, mandat, dictateur, merkel, 4...
16      [vladimir, poutine, longévité, pouvoir, encore...
17      [avant

In [25]:
tweetslemma = tweetslemma.apply(filtration)

# Polarité

In [27]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [31]:
pol = pd.read_csv("./mots.csv", sep=",", encoding = "ISO-8859-1", error_bad_lines=False)

In [32]:
pol.head()

#Transformer le df pol en deux listes

pP = pol[pol['polarity']=='positive'].word.values
pN = pol[pol['polarity']=='negative'].word.values

In [33]:
pN = [w.strip() for w in pN]

In [34]:
def calculpolarite(text: str):
    scoreP = [True for word in text if word in pP].count(True)
    scoreN = [True for word in text if word in pN].count(True)
    if(scoreP>scoreN):
        p = 'positif'
    elif(scoreP==scoreN):
        p= 'neutre'
    else:
        p= 'negatif'
    return p

In [35]:
polarite = list(tweetslemma.apply(calculpolarite))

In [36]:
tweets['sentiment'] = polarite
tweets.head()

,date,text,hashtags,sentiment
0,Sun Mar 18 13:45:13 +0000 2018,"[au, lieu, d'expulser, des, diplomates, russes...",NaN,positif
1,Sun Mar 18 15:47:19 +0000 2018,"[selon, lex, ambassadeur, anglais, en, ouzbéki...",NaN,positif
2,Sun Mar 18 11:48:32 +0000 2018,"[philipj37, eugeniebastie, tellement, vrai, s'...",NaN,positif
3,Sun Mar 18 15:41:24 +0000 2018,"[bombe, le, capitaine, barril, ancien, command...",NaN,neutre
4,Sun Mar 18 14:32:39 +0000 2018,"[dans, la, même, semaine, macron, nomme, le, t...","['Macron', 'Bellatar', 'ConseilDesVilles']",positif


In [37]:
# from collections import Counter

# def evaltweet(s:list):
#     pola = [' '.join(pol[pol.word == e].polarity) for e in s]
#     pola = Counter(pola)
#     polap = pola.get('positive', 0)
#     polan = pola.get('negative', 0)
#     return '¯\\_(ツ)_/¯' if (polap + polan) == 0 else {'positif':polap / (polap + polan), 'negatif':polan / (polap + polan)}

In [38]:
# def evaltweet2(s:list):
#     pola = [' '.join(pol[pol.word == e].polarity) for e in s]
#     pola = Counter(pola)
#     polap = pola.get('positive', 0)
#     polan = pola.get('negative', 0)
#     if (polap == polan):
#         p = 'neutre'
#     elif (polap>polan):
#         p= 'positif'
#     else:
#         p= 'negatif'
#     return p

In [39]:
# def help_apply():
#     # de 0 a 7328, boucler tous les 1000
#     deb = 0
#     fin = 500
#     polarite = []
#     for i in range(16):
#         polarite.append(tweetslemma[deb:fin].apply(evaltweet2))
#         deb += 500
#         fin += 500
#         if(fin>7328):
#             fin = 7328
#     return polarite

In [40]:
# tweets['processed_text'] = tweetslemma
# tweets.head()

In [41]:
tweetslemma.shape

(7327,)

## Preparation des donnees pour l'export

In [42]:
# for i in pol.word:
#     if (tweets.text[0].find(i)) != -1:
#         print("Ce tweet contient [" + i + "] avec la polarité: " + pol.polarity)

In [43]:
tweets.hashtags.replace("[][']", '', regex=True, inplace=True)

In [44]:
textjoin = lambda l : ','.join([w for w in l])
t = tweetslemma.apply(textjoin)
tweets['words'] = t

In [45]:
tweets.head()

,date,text,hashtags,sentiment,words
0,Sun Mar 18 13:45:13 +0000 2018,"[au, lieu, d'expulser, des, diplomates, russes...",NaN,positif,"lieu,expulser,diplomate,russe,plus,concret,plu..."
1,Sun Mar 18 15:47:19 +0000 2018,"[selon, lex, ambassadeur, anglais, en, ouzbéki...",NaN,positif,"selon,lex,ambassadeur,anglais,ouzbékistan,``,n..."
2,Sun Mar 18 11:48:32 +0000 2018,"[philipj37, eugeniebastie, tellement, vrai, s'...",NaN,positif,"philipj37,eugeniebastie,tellement,vrai,si,occu..."
3,Sun Mar 18 15:41:24 +0000 2018,"[bombe, le, capitaine, barril, ancien, command...",NaN,neutre,"bombe,capitaine,barril,ancien,commander,gign,«..."
4,Sun Mar 18 14:32:39 +0000 2018,"[dans, la, même, semaine, macron, nomme, le, t...","Macron, Bellatar, ConseilDesVilles",positif,"semaine,macron,nommer,très,ambigu,islamisme,be..."


In [49]:
#Recuperer la colonne text du df de depart
tweets['text'] = tw.text
tweets.head()

,date,text,hashtags,sentiment,words
0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NaN,positif,"lieu,expulser,diplomate,russe,plus,concret,plu..."
1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NaN,positif,"selon,lex,ambassadeur,anglais,ouzbékistan,``,n..."
2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NaN,positif,"philipj37,eugeniebastie,tellement,vrai,si,occu..."
3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NaN,neutre,"bombe,capitaine,barril,ancien,commander,gign,«..."
4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"Macron, Bellatar, ConseilDesVilles",positif,"semaine,macron,nommer,très,ambigu,islamisme,be..."


In [52]:
#Remplace les nan par NO HASHTAG
tweets.hashtags.replace(np.nan, 'NO HASHTAG' ,inplace=True)
tweets.head()

,date,text,hashtags,sentiment,words
0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NO HASHTAG,positif,"lieu,expulser,diplomate,russe,plus,concret,plu..."
1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NO HASHTAG,positif,"selon,lex,ambassadeur,anglais,ouzbékistan,``,n..."
2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NO HASHTAG,positif,"philipj37,eugeniebastie,tellement,vrai,si,occu..."
3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NO HASHTAG,neutre,"bombe,capitaine,barril,ancien,commander,gign,«..."
4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"Macron, Bellatar, ConseilDesVilles",positif,"semaine,macron,nommer,très,ambigu,islamisme,be..."


In [54]:
# Export en CSV
tweets.to_csv("tweetsAnalyse.csv", index=False, encoding='utf-8')